<a href="https://colab.research.google.com/github/ysikalie/colorization/blob/master/full_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import tensorflow as tf
import keras
from keras.applications.densenet import DenseNet201
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate,Dense
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

# 新段落

In [0]:
!mkdir train

In [0]:
# Get images
X = []
for filename in os.listdir('Train/'):
    X.append(img_to_array(load_img('Train/'+filename, target_size=(256, 256))))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X
print(Xtrain.shape)
X_Test = []
for filename in os.listdir('Test/'):
    X_Test.append(img_to_array(load_img('Test/'+filename,target_size=(256, 256))))
X_Test = np.array(X_Test, dtype=float)
Xtest = 1.0/255*X_Test
print(Xtest.shape)



FileNotFoundError: ignored

In [0]:
# embed_input = Input(shape=(1000,))

#Mid-Level features network
gray_encoder_input = Input(shape=(128, 128, 1,))
gray_encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_input)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)

#Add
#Global Features network
color_encoder_input = Input(shape=(128, 128, 3,))
color_encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(color_encoder_input)
color_encoder_output = BatchNormalization()(color_encoder_output)
color_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(color_encoder_output)
color_encoder_output = BatchNormalization()(color_encoder_output)
color_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(color_encoder_output)
color_encoder_output = BatchNormalization()(color_encoder_output)
color_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(color_encoder_output)
color_encoder_output = BatchNormalization()(color_encoder_output)
color_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(color_encoder_output)
color_encoder_output = BatchNormalization()(color_encoder_output)
# color_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(color_encoder_output)
# color_encoder_input = BatchNormalization()(color_encoder_input)
# color_encoder_output = Flatten(name='flatten')(color_encoder_output)
# color_encoder_output = Dense(256, activation='sigmoid')(color_encoder_output)

# #Fusion
# fusion_output = RepeatVector(32 * 32)(color_encoder_output) 
# fusion_output = Reshape(([32, 32, 256]))(fusion_output)
fusion_output = concatenate([gray_encoder_output, color_encoder_output], axis=3) 
 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)


model = Model(inputs=[gray_encoder_input, color_encoder_input], outputs=decoder_output)
model.summary()



In [0]:


# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 8

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
#         grayscaled_rgb = gray2rgb(rgb2gray(batch))
#         embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
#         print([X_batch,lab_batch], Y_batch)
        yield ([X_batch, lab_batch], Y_batch)


def test_image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtest, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
#         embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(grayscaled_rgb)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, lab_batch], Y_batch)
               
        
#Train model 
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
if os.path.exists('/home/weights-best.hdf5'):
    model.load_weights('/home/weights-best.hdf5')
    print("checkpoint_loaded")
model.compile(optimizer='rmsprop', loss='mse')
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100, min_delta=1E-7, verbose=1)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
filepath="weights-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max',period=1)
model.fit_generator(image_a_b_gen(batch_size), epochs=2, 
                    steps_per_epoch=100,
                    validation_data=test_image_a_b_gen(batch_size),
                    validation_steps=105,
                    callbacks=[checkpoint, rlrp, es])

In [0]:
color_me = []
for filename in os.listdir('/input/train/result31/'):
    color_me.append(img_to_array(load_img('/input/train/result31/'+filename,target_size=(128, 128))))
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, gray_me])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((128, 128, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("/input/train/result10/img_"+str(i)+".png", lab2rgb(cur))